<a href="https://colab.research.google.com/github/chunxu8894/tensorflow-tutorials/blob/master/TensorFlow_2_quickstart_for_experts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 2 quickstart for experts

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## 1. Load and prepare the MNIST Dataset

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0

#Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

### Use __tf.data__ to batch and shuffle the dataset:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 2. Build the model

Build the tf.keras model using the Keras model subclassing API:

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32,3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)
    
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

#Create an intance of the model
model = MyModel()
    

## 3. Optimizer and loss function

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

## 4. Metrics and accuracy

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

##. 5 Train the Model

Use the tf.GradientTape to train the model

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    #traing = True is only needed if there are layers with different behavior during training versus inference (e.g Dropout)
    predictions = model(images, training = True)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


Test the model


In [0]:
@tf.function
def test_step(images, labels):

  #training = False is only needed if there are layers with different behavior during training versus inference
  predictions = model(images, training = False)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)



In [16]:
EPOCHS = 5

for epoch in range(EPOCHS):
  
  #Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states

  for images, labels in train_ds:
    train_step(images, labels)

  for images, labels in test_ds:
    test_step(images, labels)

  template = 'Epoch {}, Loss: :{}, Accuracy: {}, TestLoss: {}, Test Accuracy: {}'
  print(template.format(epoch+1, train_loss.result(), train_accuracy.result()*100, test_loss.result, test_accuracy.result()*100))

Epoch 1, Loss: :0.13190606236457825, Accuracy: 95.97833251953125, TestLoss: <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7fdd03afdc50>>, Test Accuracy: 97.43000030517578
Epoch 2, Loss: :0.04014884680509567, Accuracy: 98.76000213623047, TestLoss: <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7fdd03afdc50>>, Test Accuracy: 97.61499786376953
Epoch 3, Loss: :0.021506162360310555, Accuracy: 99.31500244140625, TestLoss: <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7fdd03afdc50>>, Test Accuracy: 97.70333099365234
Epoch 4, Loss: :0.012634452432394028, Accuracy: 99.54499816894531, TestLoss: <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7fdd03afdc50>>, Test Accuracy: 97.81999969482422
Epoch 5, Loss: :0.009984318166971207, Accuracy: 99.66999816894531, TestLoss: <bound method Reduce.result of <tensorflow.python.keras.metrics.Mean object at 0x7fdd03afdc50>>, 

We need to discuss about some function in this tutorial such as:
- @tf.function
- How to calculate the accuracy
- The different between variant of Loss function
- What is tf.GradientTape
- SparseCategoricalCrossentropy ?
- Why #Add a channels dimension with these command
    + x_train = x_train[..., tf.newaxis]
    + x_test = x_test[..., tf.newaxis]
- Why the test_step have no tf.GradientTape